In [1]:
import os
import xlsxwriter
import import_ipynb
import polars as pl
import coingecko_api as ca
import crypto_scrape as cs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

importing Jupyter notebook from crypto_api.ipynb
importing Jupyter notebook from crypto_scrape.ipynb


In [2]:
def create_excel(path: str, data: dict):
    """Creates an Excel file with dict of dataframes"""
    with xlsxwriter.Workbook(path) as workbook:
        for key, df in data.items():
            workbook.add_worksheet(key)
            df.write_excel(workbook=workbook, worksheet=key)

In [3]:
def import_excel(path: str, names: list):
    """Create dict of dataframes from Excel sheets"""
    data = {}
    for name in names:
        data[name] = pl.read_excel(
            path, 
            sheet_name=name,
            read_csv_options={
                'infer_schema_length': 2000,
                'dtypes': {'date': pl.Date}
            }
        )
    return data

In [4]:
# List of crypto symbols and IDs
syms = ['btc', 'xrp', 'doge', 'ltc', 'xmr', 'bch', 'xlm', 'bsv', 'zec', 'dash']
cids = ['bitcoin', 'ripple', 'dogecoin', 'litecoin', 'monero', 'bitcoin-cash', 'stellar', 'bitcoin-cash-sv', 'zcash', 'dash']

# Variables for API call
currency = 'usd'
start_date = '2019/01/01'
end_date = '2022/12/31'

In [5]:
file_path = os.path.join("data/crypto/", "api-data.xlsx")

# Check if file already exists
if not os.path.exists(file_path):
    # Call the API and create dict of dataframes
    api_data = ca.create_api_dict(syms, cids, currency, start_date, end_date)
    # Create Excel file with API data
    create_excel(file_path, api_data)

In [6]:
# Set up Chrome headless options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# Set up the webdriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)

In [7]:
# Dict of token stats list for scraping
token_stats = {
    'btc': ['bitcoin-transactions', 'bitcoin-transactionvalue', 'bitcoin-mediantransactionvalue', 'bitcoin-transactionfees', 'bitcoin-median_transaction_fee'],
    'xrp': ['xrp-transactions', 'transactionfees-xrp'],
    'doge': ['dogecoin-transactions', 'dogecoin-transactionvalue', 'dogecoin-mediantransactionvalue', 'dogecoin-transactionfees', 'dogecoin-median_transaction_fee'],
    'ltc': ['litecoin-transactions', 'litecoin-transactionvalue', 'litecoin-mediantransactionvalue', 'litecoin-transactionfees', 'litecoin-median_transaction_fee'],
    'xmr': ['monero-transactions', 'monero-transactionfees'],
    'bsv': ['transactions-bsv'],
    'bch': ['bitcoin%20cash-transactions', 'bitcoin%20cash-transactionvalue', 'bitcoin%20cash-mediantransactionvalue', 'bitcoin%20cash-transactionfees', 'bitcoin%20cash-median_transaction_fee'],
    'zec': ['zcash-transactions', 'zcash-transactionfees', 'transactionvalue-zec', 'mediantransactionvalue-zec'],
    'dash': ['dash-transactions', 'dash-transactionfees', 'dash-median_transaction_fee', 'transactionvalue-dash', 'mediantransactionvalue-dash']
}

In [8]:
file_path = os.path.join("data/crypto/", "scraped-data.xlsx")

# Check if file already exists
if not os.path.exists(file_path):
    # Scrape BitInfoCharts and create dict of dataframes
    scraped_data = cs.create_scrape_dict(token_stats, driver, start_date, end_date)
    # Create Excel file with scraped data
    create_excel(file_path, scraped_data)

In [9]:
# Read filled Excel sheets into dataframes
filled_api = import_excel(
    'data/crypto/filled-api-data.xlsx', 
    syms
)
filled_scraped = import_excel(
    'data/crypto/filled-scraped-data.xlsx', 
    ['btc', 'xrp_bit', 'xrp_mes', 'doge', 'ltc', 'xmr', 'bsv_bit', 'bsv_mes', 'bch', 'xlm', 'zec', 'dash']
)

In [10]:
# Fill missing dates in Messari data with null values
for name in ['xrp_mes', 'bsv_mes', 'xlm']:
    filled_scraped[name] = ca.fill_date(filled_scraped[name])

In [11]:
# Merge XRP and BSV dataframes
for name in ['xrp', 'bsv']:
    filled_scraped[name] = filled_scraped[f'{name}_bit'].join(filled_scraped[f'{name}_mes'], on='date', how='inner')
    # Calculate average_transaction_fees column
    filled_scraped[name] = filled_scraped[name].with_columns((pl.col("total_fees") / pl.col("transactions_count")).alias("average_transaction_fees"))
    # Drop total_fees column
    filled_scraped[name] = filled_scraped[name].drop('total_fees')

In [12]:
# Merge API and scraped data
combined_data = {}
for name in syms:
    combined_data[name] = filled_api[name].join(filled_scraped[name], on='date', how='inner')

# Create Excel file with combined data
file_path = os.path.join("data/crypto/", "combined-data.xlsx")
create_excel(file_path, combined_data)